In [11]:
cuda_code = r"""
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>

#define IDX(i,j,N) ((i)*(N)+(j))

__global__ void update_kernel(double *u, double *uNew, int Nx, int Ny, 
                             double dx, double dy, double alpha, double dt) {
    int i = blockIdx.x * blockDim.x + threadIdx.x + 1;
    int j = blockIdx.y * blockDim.y + threadIdx.y + 1;
    if (i < Nx-1 && j < Ny-1) {
        // Use const variables for denominators
        const double dx2 = dx*dx;
        const double dy2 = dy*dy;
        const double factor = alpha*dt;
        
        // Calculate derivatives with fused operations
        double uxx = (u[IDX(i+1,j,Ny)] - 2.0*u[IDX(i,j,Ny)] + u[IDX(i-1,j,Ny)]) / dx2;
        double uyy = (u[IDX(i,j+1,Ny)] - 2.0*u[IDX(i,j,Ny)] + u[IDX(i,j-1,Ny)]) / dy2;
        
        // Use exact same operation order as CPU version
        uNew[IDX(i,j,Ny)] = fma(factor, (uxx + uyy), u[IDX(i,j,Ny)]);
    }
}

int main(int argc, char *argv[]) {
    int Nx = 200, Ny = 200;
    int Nt = 1000;
    double Lx = 1.0, Ly = 1.0;
    double alpha = 0.0001;
    double dx = Lx/(Nx-1), dy = Ly/(Ny-1);
    double dt = 0.25 * fmin(dx*dx, dy*dy) / alpha;
    size_t N = Nx * Ny;

    // Host allocation
    double *u = (double*)malloc(N * sizeof(double));
    double *uNew = (double*)malloc(N * sizeof(double));

    // Initial condition
    for(int i=0;i<Nx;i++){
        for(int j=0;j<Ny;j++){
            double x = i*dx - Lx/2, y = j*dy - Ly/2;
            u[IDX(i,j,Ny)] = exp(-50*(x*x + y*y));
        }
    }

    // Device allocation
    double *d_u, *d_uNew;
    cudaMalloc(&d_u, N * sizeof(double));
    cudaMalloc(&d_uNew, N * sizeof(double));

    // Copy initial data to device
    cudaMemcpy(d_u, u, N * sizeof(double), cudaMemcpyHostToDevice);

    // CUDA timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    // Kernel launch config
    dim3 block(16, 16);
    dim3 grid((Nx-2+block.x-1)/block.x, (Ny-2+block.y-1)/block.y);

    // Time-stepping
    for(int n=0; n<Nt; n++) {
        update_kernel<<<grid, block>>>(d_u, d_uNew, Nx, Ny, dx, dy, alpha, dt);
        // Swap pointers
        double *tmp = d_u; d_u = d_uNew; d_uNew = tmp;
    }
    cudaDeviceSynchronize();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms = 0;
    cudaEventElapsedTime(&ms, start, stop);
    double elapsed = ms * 1e-3;

    // Copy result back
    cudaMemcpy(u, d_u, N * sizeof(double), cudaMemcpyDeviceToHost);

    // Compute throughput
    double updates = (double)Nt*(Nx-2)*(Ny-2);
    double mlups = updates / elapsed / 1e6;
    FILE *file = fopen("cuda_heat_distribution.csv", "w");
    for(int i = 0; i < Nx; i++) {
        for(int j = 0; j < Ny; j++) {
            fprintf(file, "%.10e", u[IDX(i,j,Ny)]);
            if(j < Ny - 1) {
                fprintf(file,",");
            }
        }
        fprintf(file," \n ");
    }


    // Report
    printf("CUDA run (GPU):\\n");
    printf("  Time           : %.6f s\\n", elapsed);
    printf("  Throughput     : %.2f MLUPS\\n", mlups);
    printf("  u_center (mid) : %f\\n", u[IDX(Nx/2,Ny/2,Ny)]);

    // Cleanup
    free(u);
    free(uNew);
    cudaFree(d_u);
    cudaFree(d_uNew);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    return 0;
}
"""

with open("cuda_heat.cu", "w") as f:
    f.write(cuda_code)

In [2]:
# !nvcc -O3 -o cuda_heat cuda_heat.cu

In [12]:
!nvcc --fmad=false --prec-div=true --prec-sqrt=true -o cuda_heat cuda_heat.cu


In [13]:
!./cuda_heat

CUDA run (GPU):\n  Time           : 0.080275 s\n  Throughput     : 488.37 MLUPS\n  u_center (mid) : 0.441778\n

In [8]:
rm cuda_heat cuda_run cuda_heat_distribution.csv